In [1]:
%matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Se utilizará el conjunto de datos Breast Cancer, relacionado con el diagnóstico de cancer de pecho.

In [8]:
# Carga de los datos
#from sklearn.datasets import load_breast_cancer
#breast = load_breast_cancer()

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score 

In [9]:
#Exploración de los datos
reg = linear_model.LogisticRegression()

df = pd.read_csv("Documentsporcentajes.csv")

lstx = df[df.columns[:-6]].to_numpy()
lsty = df[df.columns[-1]].to_numpy()


In [7]:
lsty

array(['Fail', 'Fail', 'Pass', ..., 'Pass', 'Fail', 'Fail'], dtype=object)

In [7]:
# Se construye el dataframe. Se separan datos de entrada y datos de salida
#feature_names = breast['feature_names']
#breast_data = pd.DataFrame(breast['data'],columns = [feature_names])
#breast_target = pd.DataFrame(breast['target'], columns = ['Class'])
#breast_dataset = pd.concat([breast_data, breast_target], axis = 1)

In [8]:
#breast_dataset.shape

(569, 31)

In [9]:
# Se visualizan las primeras filas de datos
#breast_dataset.head()

,"(mean radius,)","(mean texture,)","(mean perimeter,)","(mean area,)","(mean smoothness,)","(mean compactness,)","(mean concavity,)","(mean concave points,)","(mean symmetry,)","(mean fractal dimension,)",...,"(worst texture,)","(worst perimeter,)","(worst area,)","(worst smoothness,)","(worst compactness,)","(worst concavity,)","(worst concave points,)","(worst symmetry,)","(worst fractal dimension,)",Class
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


Para estas primeras pruebas no se escalarán las variables.

In [13]:
# Para la estimación del modelo con regresión logística
#from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 

In [14]:
#Se preparan los datos para realizar la construcción del modelo. 
#X = breast['data']
#y = breast['target']

x_train , x_test, y_train, y_test = train_test_split(lstx, lsty)

1) Ejemplo con solo entrenamiento y test, sin regularización ni ajuste de hiperparámetros.

In [28]:
#Division entre entrenamiento y test
#X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y,random_state = 0)

In [15]:
#Se crea el modelo. Siguiendo las recomendacioens de la documentación, como la muestra es pequeña, se utilizará el método 
# de optimización liblinear el cual de basa en descenso por el gradiente.
logreg = LogisticRegression(solver = 'liblinear', random_state = 0)

In [17]:
#Se entrena el modelo
logreg.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
# Evaluación del modelo
print("Exactitud del modelo sobre el conjunto de entrenamiento: {:.3f}".format(logreg.score(x_train, y_train)))
print("Exactitud del modelo sobre el conjunto test: {:.3f}". format(logreg.score(x_test,y_test)))

Exactitud del modelo sobre el conjunto de entrenamiento: 0.628
Exactitud del modelo sobre el conjunto test: 0.627


In [48]:
#Para sacar la matriz de confusión
from sklearn.metrics import confusion_matrix

In [49]:
logreg_predicted = logreg.predict(X_test)
confusion = confusion_matrix(y_test, logreg_predicted)
print('Regresión logistica\n', confusion)

Regresión logistica
 [[49  4]
 [ 5 85]]


In [20]:
#Para mostrar un reporte con un conjunto de métricas
from sklearn.metrics import classification_report
print(classification_report(y_test, logreg_predicted, target_names = lsty))

NameError: name 'logreg_predicted' is not defined

2) Ahora se utilizará regularización L2 (definida por defecto). Se debe entonces buscar el valor del hiperparámetro de regularización.

In [21]:
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import KFold 
kf = KFold(n_splits=10, random_state = 0)
logreg = LogisticRegression(solver = 'liblinear', random_state = 0)
parametros = {'C': [0.001, 0.01, 0.05, 0.1, 1.0, 10, 100]}
mejor_modelo = GridSearchCV(logreg,param_grid =parametros, cv=kf)
mejor_modelo.fit(x_train, y_train) 

print("Mejor parámetro: {}".format(mejor_modelo.best_params_)) 
print("Mejor cross-validation score: {:.2f}".format(mejor_modelo.best_score_))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Mejor parámetro: {'C': 100}
Mejor cross-validation score: 0.64


In [22]:
# Error sobre el conjunto test
print("Test set score: {:.2f}".format(mejor_modelo.score(x_test, y_test)))

Test set score: 0.64


¿Los resultados cambiarían si se escalan o normalizan los datos?
¿Qué resultado se obtendría si se utiliza regularización L1? 
¿Cómo visualizar la importancia de las variables a través del valor de los coeficientes?